In [3]:
# 쥬피터에선 안되고 파이썬에서만 됨...
# tensorboard --logdir=./logs
# 를 실행하면 텐서보드 실행됨 http://localhost:6006

import tensorflow as tf
import numpy as np

# np.loadtxt : 데이터를 읽어 들임
data = np.loadtxt('./data.csv', delimiter = ',',
                 unpack=True, dtype= 'float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

## 신경망 모델 구성
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# with tf.name_scope 를 묶은 블록은 텐서보드에서 한 계층 내부를 표현해줌
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name ='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name ='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('ourput'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name ='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels =Y , logits = model))

    
optimizer = tf.train.AdamOptimizer(learning_rate =0.01)
# global_step = global_step 을 통해 최적화 함수가 학습용 변수들을 최적화할 떄마다 global_step 변수의 값을 1씩 증가시킴
train_op = optimizer.minimize(cost, global_step = global_step)

# 손실값을 추적하기 위해 수집할 값을 지정
# tf.summary 모듈의 scalar 함수는 값이 하나인 텐서를 수집할 떄 사용
tf.summary.scalar('cost', cost)

# tf.reset_default_graph()
## 신경망 모델 학습
sess = tf.Session()
# tf.global_variables 는 앞서 정의한 변수들을 가져오는 함수 
saver = tf.train.Saver(tf.global_variables())

# ./model 디렉토리에 학습된 모델이 있는지 확인하고 모델이 있다면 saver.restore 함수를 통해 학습된 값들을 불러오거나 
# 변수를 새로 초기화 함
ckpt = tf.train.get_checkpoint_state('./model')
print(ckpt)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
# print(ckpt.model_checkpoint_path)
# sess.run(tf.global_variables_initializer())

# mertge_all 로 지정한 텐서들을 수집한 다음, FileWriter 함수를 이용해 그래프와 텐서들의 값을 저장할 디렉터릴를 설정
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
         'Cost: %.3f ' % sess.run(cost, feed_dict={X:x_data, Y: y_data}))

# 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트에 저장
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

# 결과 확인
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy *100, feed_dict={X: x_data, Y: y_data}))

None
Step: 1,  Cost: 1.004 
Step: 2,  Cost: 0.956 
Step: 3,  Cost: 0.911 
Step: 4,  Cost: 0.870 
Step: 5,  Cost: 0.832 
Step: 6,  Cost: 0.798 
Step: 7,  Cost: 0.767 
Step: 8,  Cost: 0.740 
Step: 9,  Cost: 0.714 
Step: 10,  Cost: 0.691 
Step: 11,  Cost: 0.672 
Step: 12,  Cost: 0.654 
Step: 13,  Cost: 0.639 
Step: 14,  Cost: 0.626 
Step: 15,  Cost: 0.615 
Step: 16,  Cost: 0.605 
Step: 17,  Cost: 0.597 
Step: 18,  Cost: 0.590 
Step: 19,  Cost: 0.585 
Step: 20,  Cost: 0.580 
Step: 21,  Cost: 0.575 
Step: 22,  Cost: 0.572 
Step: 23,  Cost: 0.569 
Step: 24,  Cost: 0.566 
Step: 25,  Cost: 0.564 
Step: 26,  Cost: 0.562 
Step: 27,  Cost: 0.561 
Step: 28,  Cost: 0.560 
Step: 29,  Cost: 0.558 
Step: 30,  Cost: 0.557 
Step: 31,  Cost: 0.557 
Step: 32,  Cost: 0.556 
Step: 33,  Cost: 0.555 
Step: 34,  Cost: 0.555 
Step: 35,  Cost: 0.554 
Step: 36,  Cost: 0.554 
Step: 37,  Cost: 0.553 
Step: 38,  Cost: 0.553 
Step: 39,  Cost: 0.553 
Step: 40,  Cost: 0.552 
Step: 41,  Cost: 0.552 
Step: 42,  Cost: 0.5